In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

url = 'https://raw.githubusercontent.com/Alcalyxz/datasetProjectIA/main/Training-Dataset.csv'

df = pd.read_csv(url)
print(df.head(5))


   having_IP_Address  URL_Length  ...  Statistical_report  Result
0                 -1           1  ...                  -1      -1
1                  1           1  ...                   1      -1
2                  1           0  ...                  -1      -1
3                  1           0  ...                   1      -1
4                  1           0  ...                   1       1

[5 rows x 31 columns]


In [5]:
df = df.drop(['having_Sub_Domain'], axis=1)
df = df.drop(['URL_of_Anchor'], axis=1)
df = df.drop(['Links_in_tags'], axis=1)
df = df.drop(['Redirect'], axis=1)
df = df.drop(['web_traffic'], axis=1)
df = df.drop(['Links_pointing_to_page'], axis=1)

In [6]:
df = df.drop(df[df['URL_Length']==0].index)
df = df.drop(df[df['SSLfinal_State']==0].index)
df = df.drop(df[df['SFH']==0].index)

In [7]:
YEntrenamiento = df['Result']
XEntrenamiento = scaler.fit_transform(df.drop(['Result'],axis=1))

In [8]:
def ErrorClas(Y_lest, Y):
    error = 1 - np.sum(Y_lest == Y)/len(Y)
    return error

In [9]:
def KNN_Clasificacion(X_train, Y_train, X_test, k):
    if k > X_train.shape[0]:
        print("k no puede ser menor que las muestras de entrenamiento")
        return(None)
    distancias = scipy.spatial.distance_matrix(X_test, X_train) ##Calculo la distancia de las dos matrices... mismos elementos
    Yest = np.zeros(X_test.shape[0])
    
    for i in range (len(X_test)):
        index_vecinos = np.argsort(distancias[i])[:k]  #con argsot la ordeno con los vecinos más cercanos, me retorna el índice de los menores.
        k_vecinos = np.take(Y_train, index_vecinos)  ## Acá me traigo el valor de la matriz Yv, con el índice de los vecinos que obtuve arriba
        Yest[i] = stats.mode(k_vecinos)[0]   ##Como la función directamente me dice que es de clasificación, entonces hago solo la moda.

        
    
    return (Yest, distancias)

In [10]:

def train_test_split_fix(X,Y):
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, train_size = 0.7, test_size = 0.3 )

    return (Xtrain, Xtest, Ytrain, Ytest)

In [11]:

from scipy.spatial import distance

In [12]:
def kernel_gaussiano(x):
    return np.exp((-0.5)*x**2)

def ParzenWindow(x,Data,h):
    h = h
    Ns = Data.shape[0]
    suma = 0
    for k in range(Ns):
        u = distance.euclidean(x,Data[k,:])
        suma += kernel_gaussiano(u/h)
    return suma

In [13]:

def parzenClass(X_train, Y_train, X_test, h):
        
    Yest = np.zeros(X_test.shape[0])
    clases = np.unique(Y_train)
    fds_matrix = np.zeros((X_test.shape[0], len(clases)))
    
    index = 0
    for n, sample in enumerate (X_test):
      max_prob = 0
      est_label = None
      j = 0
      for label in clases:
        indixes = np.argwhere(Y_train == label)
        train_elements = np.take(X_train, indixes.T[0], axis = 0)
        dim = train_elements.shape
        prob = ParzenWindow(sample,train_elements,h)/dim[0]
        fds_matrix[index][j] = prob
        if prob > max_prob:
          max_prob = prob
          est_label = label
        j += 1
      Yest[index] = est_label
      index +=1
    
    return Yest, fds_matrix

In [14]:
from sklearn.model_selection import StratifiedKFold

def experimentarParzen (X, Y, hs):
    
    skf = StratifiedKFold(n_splits=4)
    resultados = pd.DataFrame()
    idx = 0
    for h in hs:
        # lista para almacenar los errores de cada iteración
        # de la validación
        error_temp = []
        
        
        for train, test in skf.split(X, Y):

            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #normalizamos los datos
            scaler = StandardScaler()
            scaler.fit(Xtrain)
            Xtrain = scaler.transform(Xtrain)
            Xtest = scaler.transform(Xtest)
            
            Yest, probabilidades = parzenClass(Xtrain, Ytrain, Xtest, h)
            errorTest = ErrorClas(Yest, Ytest)
            error_temp.append(errorTest)
    
        resultados.loc[idx,'ancho de ventana'] = h 
        resultados.loc[idx,'error de prueba(media)'] = errorTest.mean()
        resultados.loc[idx,'error de prueba(desviación estandar)'] = errorTest.std()
        idx+=1
    return (resultados)

In [15]:
YEntrenamiento = np.array(YEntrenamiento)
YEntrenamiento = YEntrenamiento.astype(int)


In [ ]:
hs = [0.05, 0.02, 0.001, 0.001, 2, 5, 10]
experimentos_parzen = experimentarParzen(XEntrenamiento,YEntrenamiento, hs)
experimentos_parzen

,ancho de ventana,error de prueba(media),error de prueba(desviación estandar)
0,0.050,0.620053,0.0
1,0.020,0.620053,0.0
2,0.001,0.620053,0.0
3,0.001,0.620053,0.0
4,2.000,0.127089,0.0
5,5.000,0.149956,0.0
6,10.000,0.148637,0.0
